In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
# data visualization
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
# display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# import dataset
df = pd.read_csv('data_repository.csv')
scan_detail = pd.read_csv('scan_detail.csv')

## Exclude international orders

In [3]:
## exclude international order, only consider packages within the U.S.A
international=['UPS Worldwide Express','UPS Worldwide Saver','UPS Worldwide Express Saver','UPS Worldwide Expedited',
             'FedEx International Economy Freight','International Priority','FedEx International Ground',
            'FedEx International Economy','FedEx International First']
# international list 1
list1=['GUADALAJARA', 'DUBAI','TLAJOMULCO DE ZUNIGA', 'BOTANY', 'GLOSTRUP', 'COFFS HARBOUR','ENFIELD', 'ST. THOMAS', 
       'TSUEN WAN', 'STOCKHOLM-ARLANDA']
# list 2 to fill city and state
list2=['ERIE DISTRIBUTION CENTER','ALTOONA DISTRIBUTION CENTER','ATLANTA NORTH METRO DISTRIBUTION CENTER',
       'PORTLAND DISTRIBUTION CENTER ANNEX','DENVER CO  DISTRIBUTION CENTER','WEST FARGO NETWORK DISTRIBUTION CENTER']

In [4]:
# exclude international order,none travel time
df=df[~df.type_of_service.isin(international)]
df=df[~df['destination_state'].isna()]
df=df[~df['origin_city'].isin(list1)]
# exclude travel time is na
df=df[df['travel_time']!='0 days 00:00:00.000000000']
df=df[~df['travel_time'].isna()]
# origin city is NA, destination state is NA
df=df[df['origin_city']!='NONE N']
df=df[df['destination_state']!='None']
df=df[df['origin_state']!='None']

In [5]:
# function to fill state and city
def state(x):
    if (len(x.split(' '))>3) and (x not in list2):
        if x.split(' ')[-3]=='NETWORK' or x.split(' ')[-3]=='INTERNATIONAL' or x.split(' ')[-1]=='ANNEX':
            return x.split(' ')[-4]
        elif 'PROCESSING CENTER' or 'DISTRIBUTION CENTER' in x:
            return x.split(' ')[-3]
    elif x == 'ERIE DISTRIBUTION CENTER' or x == 'ALTOONA DISTRIBUTION CENTER':
        return 'PA'
    elif x == 'ATLANTA NORTH METRO DISTRIBUTION CENTER':
        return 'GA'
    elif x== 'PORTLAND DISTRIBUTION CENTER ANNEX':
        return 'OR'
    elif x== 'DENVER CO  DISTRIBUTION CENTER':
        return 'CO'
    elif x== 'WEST FARGO NETWORK DISTRIBUTION CENTER':
        return 'ND'

def city(x):
    if (len(x.split(' '))>3) and (x not in list2):
        if x.split(' ')[-3]=='NETWORK' or x.split(' ')[-3]=='INTERNATIONAL' or x.split(' ')[-1]=='ANNEX':
            return ' '.join(x.split(' ')[0:-4])
        elif 'PROCESSING CENTER' or 'DISTRIBUTION CENTER' in x:
            return' '.join(x.split(' ')[0:-3])
    elif x== 'WEST FARGO NETWORK DISTRIBUTION CENTER':
        return 'WEST FARGO'
    elif x== 'DENVER CO  DISTRIBUTION CENTER':
        return 'DENVER'
    elif x in list2 and x !='WEST FARGO NETWORK DISTRIBUTION CENTER':
        return x.split(' ')[0]
    else:
        return x

In [6]:
df['origin_state']=df['origin_state'].fillna(df['origin_city'].apply(lambda x: state(x)))
df['origin_city'] = df['origin_city'].apply(lambda x: city(x))

In [7]:
# export index csv and processed_data_repository.csv
tracking_id=df['tracking_ID']
tracking_id.to_csv('index.csv')
df.to_csv('processed_data_repository.csv')

## USPS Data Pre-Processing

In [8]:
# import index csv
index=pd.read_csv('index.csv',header=None)
index=index[1].tolist()
USPS = scan_detail[scan_detail['source']=='USPS']

In [9]:
# use index to filter international orders and Non-travel time records
USPS=USPS[USPS['tracking_ID'].isin(index)]
USPS=USPS.iloc[::-1].reset_index().drop(['index','Unnamed: 0'],axis=1)

### Deal with missing value

In [10]:
# fill missing value in datetime column
USPS['datetime'] = USPS['datetime'].fillna('January 1, 1999 01:00 am')
# change missing value to the close time
for i in range(len(USPS['datetime'])):
    if USPS['datetime'].iloc[i] == '1999-01-01 01:00:00': 
        USPS['datetime'].iloc[i] = USPS['datetime'].iloc[i-1] 
# generate Datetime column and sort the dataframe by time
USPS['DateTime'] = USPS['datetime'].apply(lambda x: datetime.strptime(x, "%B %d, %Y %I:%M %p"))
USPS['DateTime']=pd.to_datetime(USPS['DateTime'])
# sort dataframe by datetime
USPS=USPS.sort_values(['tracking_ID', 'DateTime'])

In [11]:
# fill missing value for city and generate am/pm and day_of_week columns
USPS['am/pm']=USPS['datetime'].str[-2:]
USPS['day_of_week']=USPS['DateTime'].apply(lambda x:x.isoweekday())
USPS['city'] = USPS['city'].fillna('unknown')
for i in range(len(USPS['city'])):
    if USPS['city'].iloc[i] == 'unknown': 
        USPS['city'].iloc[i] = USPS['city'].iloc[i-1]

In [12]:
# fill missing value in state and regulate state
USPS['state']=USPS['state'].fillna(USPS['city'].apply(lambda x: state(x)))
USPS['city'] = USPS['city'].apply(lambda x: city(x))

In [13]:
# fill other missing value in state
USPS['state'] = USPS['state'].fillna('unknown')
for i in range(len(USPS['state'])):
    if USPS['state'].iloc[i] == 'unknown':
        USPS['state'].iloc[i] = USPS['state'].iloc[i-1]

In [14]:
# drop columns
USPS = USPS.drop(['message','status','status_detail','country','zip'],axis=1)

In [15]:
# sanity check
USPS.isna().sum()

tracking_ID    0
datetime       0
status_code    0
description    0
source         0
city           0
state          0
DateTime       0
am/pm          0
day_of_week    0
dtype: int64

### Regulate status code

In [16]:
# delete unneeded status code
keep_status=['03','OF', '07', '10', 'A1', 'T1', 'OA',  'EF', 'SF', 'U1', 
             'L1', 'RB','15',  '08',  'UA',  'AH', 'AP','BB']
USPS=USPS[USPS.status_code.isin(keep_status)]
# regulate status code
USPS['status_code_cal'] = USPS['status_code']
USPS['status_code_cal'] = USPS['status_code_cal'].replace('07', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('A1', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('OA', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('U1', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('03', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('UA', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('AP', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('BB', 'ARR')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('OF', 'DEP')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('T1', 'DEP')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('EF', 'DEP')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('SF', 'DEP')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('L1', 'DEP')
USPS['status_code_cal'] = USPS['status_code_cal'].replace('AH', 'DEP')

In [17]:
USPS.loc[USPS['status_code_cal'].isin(['10','RB','08','15']),'status_code_cal']=None
# function to change status for specific code
def status(x):
    if (x.split(' ')[0]=='Departed') or (x.split(' ')[0]=='Dispatched'):
        return 'DEP'
    elif x.split(' ')[0]=='Arrived':
        return 'ARR'
USPS['status_code_cal']=USPS['status_code_cal'].fillna(USPS['description'].apply(lambda x: status(x)))

### Calculate time difference

In [18]:
def calculate_time(test):
    last_index=test.iloc[-1:].index[0]
    df=pd.DataFrame()
    for index,row in test.iterrows():
        if index!=last_index:
            if row['status_code_cal']=='ARR' and test.loc[index+1]['status_code_cal']=='DEP':
                if row['city']==test.loc[index+1]['city']:
                    time_spent=(test.loc[index+1]['DateTime']-row['DateTime']).total_seconds()/60
                    city=row['city']
                    state=row['state']
                    am_pm=row['am/pm']
                    day_of_week=row['day_of_week']
                    newrow=pd.Series([row['tracking_ID'],city,state,time_spent,am_pm,day_of_week])
                    df=df.append(newrow,ignore_index=True)
    return df

In [19]:
USPS.DateTime=pd.to_datetime(USPS.DateTime)
USPS = USPS.reset_index()
USPS_id = list(USPS['tracking_ID'].unique())
facility_time_usps = pd.DataFrame()
for i in USPS_id:
    test = USPS[USPS['tracking_ID'] == i]
    subdf = calculate_time(test)
    facility_time_usps = pd.concat([facility_time_usps,subdf],ignore_index=True)

In [20]:
# calculate stops
facility_time_usps=facility_time_usps.rename(columns={0:'tracking_id',1:'city',2:'state',3:'time',4:'am_pm',5:'day_of_week'},inplace=False)
stops=facility_time_usps.tracking_id.value_counts().rename_axis('tracking_id').reset_index(name='counts')
facility_time_usps=pd.merge(facility_time_usps,stops,how='left',on='tracking_id')

In [21]:
facility_time_usps.columns =['tracking_id','city','state','time','am_pm','day_of_week','stops']

In [22]:
facility_time_usps.to_csv('facility_time_usps.csv')

## UPS Data Pre-Processing

In [23]:
# get ups data
sd_ups = scan_detail[scan_detail['source']=='UPS']

In [24]:
# use index to filter international orders and Non-travel time records
sd_ups=sd_ups[sd_ups['tracking_ID'].isin(index)]

### Deal with missing value

In [25]:
sd_ups['date']=sd_ups['datetime'].str.split('T').str[0]
sd_ups['time'] = sd_ups['datetime'].str.split('T').str[1]
sd_ups['time'] = sd_ups['time'].str[0:-4]
sd_ups['new_time'] = sd_ups['time'].str.split(':').str[0]
sd_ups['am/pm'] = sd_ups.new_time.apply(lambda x: 'pm' if int(x) >= 12 else 'am')
sd_ups = sd_ups.drop(columns = ['new_time'])
sd_ups.head()

,Unnamed: 0,tracking_ID,datetime,status_code,description,message,source,status,status_detail,city,state,country,zip,date,time,am/pm
824414,824414,UPS_85776,2020-05-12T14:10:52Z,M,Order Processed: Ready for UPS,Order Processed: Ready for UPS,UPS,pre_transit,NaN,NaN,NaN,US,NaN,2020-05-12,14:10,pm
824415,824415,UPS_85776,2020-05-12T17:17:11Z,P,Pickup Scan,Pickup Scan,UPS,in_transit,NaN,Kansas City,KS,US,NaN,2020-05-12,17:17,pm
824416,824416,UPS_85776,2020-05-12T17:40:14Z,OS,Origin Scan,Origin Scan,UPS,in_transit,NaN,Kansas City,KS,US,NaN,2020-05-12,17:40,pm
824417,824417,UPS_85776,2020-05-13T0:01:00Z,OF,Departed from origin Facility,Departed from Facility,UPS,in_transit,NaN,Kansas City,KS,US,NaN,2020-05-13,0:01,am
824418,824418,UPS_85776,2020-05-13T0:26:00Z,I,Arrived at Destination Facility,Arrived at Facility,UPS,in_transit,NaN,Lenexa,KS,US,NaN,2020-05-13,0:26,am


In [26]:
sd_ups.to_csv('scan_detail_UPS.csv')

### Regulate status code

In [27]:
ups_detail = pd.read_csv('scan_detail_ups.csv')
ups = ups_detail.loc[(ups_detail['status_code'] == 'T1')|
                     (ups_detail['status_code'] == 'F1')|
                     (ups_detail['status_code'] == 'T2')|
                     (ups_detail['status_code'] == 'F2')| 
                     (ups_detail['status_code'] == 'T3')| 
                     (ups_detail['status_code'] == 'F3')|
                     (ups_detail['status_code'] == 'T4')| 
                     (ups_detail['status_code'] == 'F4')| 
                     (ups_detail['status_code'] == 'T5')|
                     (ups_detail['status_code'] == 'F5')|
                     (ups_detail['status_code'] == 'T6')|
                     (ups_detail['status_code'] == 'F6')|
                     (ups_detail['status_code'] == 'I')|
                     (ups_detail['status_code'] == 'OF')|
                     (ups_detail['status_code'] == 'OD')]

In [28]:
ups['status_code_cal'] = ups['status_code']
ups['status_code_cal'] = ups['status_code_cal'].replace('T1', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('T2', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('T3', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('T4', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('T5', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('T6', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('I', 'ARR')
ups['status_code_cal'] = ups['status_code_cal'].replace('F1', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('F2', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('F3', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('F4', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('F5', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('F6', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('OF', 'DEP')
ups['status_code_cal'] = ups['status_code_cal'].replace('OD', 'DEP')
ups.head()

,Unnamed: 0,Unnamed: 0.1,tracking_ID,datetime,status_code,description,message,source,status,status_detail,city,state,country,zip,date,time,am/pm,status_code_cal
3,824417,824417,UPS_85776,2020-05-13T0:01:00Z,OF,Departed from origin Facility,Departed from Facility,UPS,in_transit,NaN,Kansas City,KS,US,NaN,2020-05-13,0:01,am,DEP
4,824418,824418,UPS_85776,2020-05-13T0:26:00Z,I,Arrived at Destination Facility,Arrived at Facility,UPS,in_transit,NaN,Lenexa,KS,US,NaN,2020-05-13,0:26,am,ARR
7,824421,824421,UPS_85776,2020-05-13T9:14:51Z,OD,Out For Delivery Today,Out For Delivery Today,UPS,in_transit,NaN,Lenexa,KS,US,NaN,2020-05-13,9:14,am,DEP
11,824425,824425,UPS_85777,2020-06-23T20:32:00Z,OF,Departed from origin Facility,Departed from Facility,UPS,in_transit,NaN,West Yarmouth,MA,US,NaN,2020-06-23,20:32,pm,DEP
12,824426,824426,UPS_85777,2020-06-23T22:52:00Z,T1,Arrived at Facility 1,Arrived at Facility,UPS,in_transit,NaN,Shrewsbury,MA,US,NaN,2020-06-23,22:52,pm,ARR


In [29]:
ups = ups.drop('datetime',axis=1)
ups['DateTime'] = ups['date'] +' '+ ups['time']
ups['DateTime'] = pd.to_datetime(ups['DateTime'])
ups['day_of_week']=ups['DateTime'].apply(lambda x:x.isoweekday())
ups = ups.reset_index()

### Calculate time difference

In [30]:
ups_id = list(ups['tracking_ID'].unique())
facility_time_ups = pd.DataFrame()
for i in ups_id:
    test = ups[ups['tracking_ID'] == i]
    subdf = calculate_time(test)
    facility_time_ups = pd.concat([facility_time_ups,subdf],ignore_index=True)

In [31]:
# # calculate stops
facility_time_ups=facility_time_ups.rename(columns={0:'tracking_id',1:'city',2:'state',3:'time',4:'am_pm',5:'day_of_week'},inplace=False)
stops=facility_time_ups.tracking_id.value_counts().rename_axis('tracking_id').reset_index(name='counts')
facility_time_ups=pd.merge(facility_time_ups,stops,how='left',on='tracking_id')

In [32]:
facility_time_ups.columns =['tracking_id','city','state','time','am_pm','day_of_week','stops']

In [33]:
facility_time_ups.to_csv('facility_time_ups.csv')

## FedEx Data Pre-Processing

In [34]:
sd_fedex = scan_detail[scan_detail['source']=='FedEx']

In [35]:
# use index to filter international orders and Non-travel time records
sd_fedex=sd_fedex[sd_fedex['tracking_ID'].isin(index)]
sd_fedex=sd_fedex.iloc[::-1].reset_index().drop(['index','Unnamed: 0'],axis=1)

### Deal with missing value

In [36]:
sd_fedex['date']=sd_fedex['datetime'].str.split('T').str[0]
sd_fedex['time'] = sd_fedex['datetime'].str.split('T').str[1]
sd_fedex['time'] = sd_fedex['time'].str[0:-4]
sd_fedex['time'] = sd_fedex['time'].replace('', '00:00')
for i in range(len(sd_fedex['time'])):
    if sd_fedex['time'].iloc[i] == '00:00':
        sd_fedex['time'].iloc[i] = sd_fedex['time'].iloc[i-1]
sd_fedex['new_time'] = sd_fedex['time'].str.split(':').str[0]
sd_fedex['am/pm'] = sd_fedex.new_time.apply(lambda x: 'pm' if int(x) >= 12 else 'am')
sd_fedex = sd_fedex.drop(columns = ['new_time'])

In [37]:
sd_fedex = sd_fedex.drop('datetime',axis=1)
sd_fedex['DateTime'] = sd_fedex['date'] +' '+ sd_fedex['time']
sd_fedex['DateTime'] = pd.to_datetime(sd_fedex['DateTime'])
sd_fedex['day_of_week']=sd_fedex['DateTime'].apply(lambda x:x.isoweekday())
sd_fedex = sd_fedex.reset_index()

In [38]:
sd_fedex.head()

,index,tracking_ID,status_code,description,message,source,status,status_detail,city,state,country,zip,date,time,am/pm,DateTime,day_of_week
0,0,FedEx_137283,DL,Delivered,NaN,FedEx,NaN,NaN,Miami,FL,US,33137,2020-06-24,12:19,pm,2020-06-24 12:19:00,3
1,1,FedEx_137283,OD,On FedEx vehicle for delivery,NaN,FedEx,NaN,NaN,MIAMI,FL,US,33178,2020-06-24,08:55,am,2020-06-24 08:55:00,3
2,2,FedEx_137283,OD,On FedEx vehicle for delivery,NaN,FedEx,NaN,NaN,MIAMI,FL,US,33178,2020-06-17,05:41,am,2020-06-17 05:41:00,3
3,3,FedEx_137283,AR,At local FedEx facility,NaN,FedEx,NaN,NaN,MIAMI,FL,US,33178,2020-06-17,05:17,am,2020-06-17 05:17:00,3
4,4,FedEx_137283,OD,On FedEx vehicle for delivery,NaN,FedEx,NaN,NaN,MIAMI,FL,US,33178,2020-06-16,07:52,am,2020-06-16 07:52:00,2


In [39]:
# sd_fedex.to_csv('scan_detail_FedEx.csv')

### Regulate status code

In [40]:
# use index to filter international orders and Non-travel time records
Fedex=sd_fedex[sd_fedex['tracking_ID'].isin(index)]
# filter out some status code
Fedex=Fedex[(Fedex['status_code']!='SH') & (Fedex['status_code']!='OC')]

In [41]:
Fedex['DateTime']=pd.to_datetime(Fedex['DateTime'])
# sort dataframe by datetime
Fedex=Fedex.sort_values(['tracking_ID', 'DateTime'])

In [42]:
#filter missing values in city and state
Fedex['state'] = Fedex['state'].fillna('unknown')
Fedex['city'] = Fedex['city'].fillna('unknown')
for i in range(len(Fedex['state'])):
    if Fedex['state'].iloc[i]=='unknown':
        Fedex['state'].iloc[i] = Fedex['state'].iloc[i-1]
for i in range(len(Fedex['city'])):
    if Fedex['city'].iloc[i]=='unknown':
        Fedex['city'].iloc[i] = Fedex['state'].iloc[i-1]

### Calculate time difference

In [43]:
# calculate day_of_week
Fedex['day_of_week']=Fedex['DateTime'].apply(lambda x:x.isoweekday())

In [44]:
def calculate_time_fedex(test):
    last_index=test.iloc[-1:].index[0]
    df=pd.DataFrame()
    for index,row in test.iterrows():
        if index!=last_index:
            if row['status_code']=='AR' and test.loc[index+1]['status_code']!='AR':
                if row['city']==test.loc[index+1]['city']:
                    time_spent=(test.loc[index+1]['DateTime']-row['DateTime']).total_seconds()/60
                    city=row['city']
                    state=row['state']
                    am_pm=row['am/pm']
                    day_of_week=row['day_of_week']
                    newrow=pd.Series([row['tracking_ID'],city,state,time_spent,am_pm,day_of_week])
                    df=df.append(newrow,ignore_index=True)
    return df

In [45]:
facility_time_fedex=pd.DataFrame()
Fedex = Fedex.reset_index()
for i in Fedex['tracking_ID'].unique():
    test=Fedex[Fedex['tracking_ID']==i]
    subdf=calculate_time_fedex(test)
    facility_time_fedex=pd.concat([facility_time_fedex,subdf],ignore_index=True)

In [46]:
# calculate stops
facility_time_fedex=facility_time_fedex.rename(columns={0:'tracking_id',1:'city',2:'state',3:'time',4:'am_pm',5:'day_of_week'},inplace=False)
stops=facility_time_fedex.tracking_id.value_counts().rename_axis('tracking_id').reset_index(name='counts')
facility_time_fedex=pd.merge(facility_time_fedex,stops,how='left',on='tracking_id')

In [47]:
facility_time_fedex.columns =['tracking_id','city','state','time','am_pm','day_of_week','stops']

In [48]:
facility_time_fedex.to_csv('facility_time_fedex.csv')